In [1]:
import numpy as np
import imageio

In [6]:
'''
getCameraParam: Get the camera matrix C
Input: choose between color camera or depth camera
Output: Intrinsic camera matrix
'''
def getCameraParam(colorOrZ = "color"):
    if(colorOrZ == "color"):
        fx_rgb = 5.1885790117450188e+02
        fy_rgb = 5.1946961112127485e+02
        cx_rgb = 3.2558244941119034e+02
        cy_rgb = 2.5373616633400465e+02
        C = np.array([[fx_rgb, 0, cx_rgb], [0, fy_rgb, cy_rgb], [0, 0, 1]])
    else:
        fx_d = 5.8262448167737955e+02
        fy_d = 5.8269103270988637e+02
        cx_d = 3.1304475870804731e+02
        cy_d = 2.3844389626620386e+02
        C = np.array([[fx_d, 0, cx_d], [0, fy_d, cy_d], [0, 0, 1]])
    return C

In [214]:
'''
getPointCloudFromZ: get position in real world
Input: Z is in cm
       C: camera matrix
       s: factor by which Zhas ben upsampled
'''
def getPointCloudFromZ(Z,C,s = 1):
    height, width= Z.shape
    xx,yy = np.meshgrid(np.array(range(width))+1, np.array(range(height))+1)
    # color camera parameters
    cc_rgb = C[0:2,2] * s
    fc_rgb = np.diag(C[0:2,0:2]) * s
    x3 = np.multiply((xx - cc_rgb[0]), Z) / fc_rgb[0]
    y3 = np.multiply((yy - cc_rgb[1]), Z) / fc_rgb[1]
    z3 = Z
    return x3, y3 ,z3

In [32]:
import scipy.io as spio

mat = spio.loadmat('data_z.mat', squeeze_me=True)

z = mat['Z'] # array
C = np.array([  [518.8579   ,      0  ,325.5824],
         [0 , 519.4696 , 253.7362],
         [0   ,      0   , 1.0000]])
x,y,z = getPointCloudFromZ(z,C)

(2,)
(2,)


In [215]:
'''
getRMatrix: Generate a rotation matrix that 
            if yf is a scalar, rotates about axis yi by yf degrees
            if yf is an axis, rotates yi to yf in the direction given by yi x yf
Input: yi is an axis 3x1 vector
       yf could be a scalar of axis
       
'''
    
def getRMatrix(yi, yf):
    if(np.isscalar(yf)):
        ax = yi/norm(yi)
        phi = yf
    else:
        yi = yi/np.linalg.norm(yi)
        yf = yf/np.linalg.norm(yf)
        ax = np.cross(yi.T,yf.T).T
        ax = ax / np.linalg.norm(ax)
        # find angle of rotation
        phi = np.degrees(np.arccos(np.dot(yi.T, yf)))

    if(abs(phi) > 0.1):
        phi = phi *(np.pi / 180)


        s_hat = np.array([[0,-ax[2],ax[1]],
                         [ax[2], 0, -ax[0]],
                         [-ax[1],ax[0],0]])
        R = np.eye(3) + np.sin(phi)*s_hat + (1-np.cos(phi)) * np.dot(s_hat,s_hat)
    else:
        R = np.eye(3)
    return R
    

In [59]:
yi = np.array([[0 ,1, 0]]).T
yf = np.array([[-0.0104,0.9431,0.3323]]).T
r = getRMatrix(yi,yf)
r

[[0.00000000e+00 9.78545020e-04 0.00000000e+00]
 [9.78545020e-04 0.00000000e+00 9.99021455e-01]
 [0.00000000e+00 9.99021455e-01 0.00000000e+00]]


array([[ 0.99994434, -0.01040016,  0.0017786 ],
       [ 0.01040016,  0.94311459, -0.33230514],
       [ 0.0017786 ,  0.33230514,  0.94317026]])

In [216]:
from scipy import signal
import cv2

def filterItChopOff(f, r, sp):
    f[np.isnan(f)] = 0
    H,W,d = f.shape
    B = np.ones([2*r+1,2*r+1])    
    kernel = np.ones((5,5), np.uint8)
    minSP = cv2.erode(sp, B, iterations = 1)
    maxSP = cv2.dilate(sp, B, iterations = 1)
    
    ind = np.where(np.logical_or(minSP!=sp, maxSP!=sp))
    spInd = sp
    spInd = range(np.size(sp))

    delta = np.zeros(f.shape)
    delta = np.reshape(delta, (H*W, d))
    f = np.reshape(f, (H*W, d))

    # calculate delta
    
    I, J = np.unravel_index(ind, [H,W], 'C')
    for i in range(np.size(ind)):
        x = I[i]
        y = J[i]
        clipInd = spInd[max(1, x-r):min(H, x+r), max(1, y-r):min(W, y+r)]
        diffInd = clipInd[sp(clipInd) != sp(x,y)]
        delta[ind[i], :] = np.sum(f[diffInd, :],1)

    delta = np.reshape(delta, (H, W, d))
    f = np.reshape(f, (H, W, d))
    fFilt = np.zeros([H,W,d]) 
    for i in range(f.shape[2]):
        #  fFilt(:,:,i) = filter2(B, f(:,:,i));
        tmp = signal.convolve2d(np.rot90(f[:,:,i],2), np.rot90(np.rot90(B,2),2), mode="same")
        fFilt[:,:,i] = np.rot90(tmp, 2)
    fFilt = fFilt-delta
    return fFilt

In [217]:
'''
Input: AtA: W x H x 6
'''
def invertIt(AtA):
    AtA_1 = np.zeros([AtA.shape[0],AtA.shape[1],6])
    AtA_1[:,:,0] = np.multiply(AtA[:,:,3], AtA[:,:,5]) - np.multiply(AtA[:,:,4], AtA[:,:,4])
    AtA_1[:,:,1] = -np.multiply(AtA[:,:,1], AtA[:,:,5]) + np.multiply(AtA[:,:,2], AtA[:,:,4])
    AtA_1[:,:,2] = np.multiply(AtA[:,:,1], AtA[:,:,4]) - np.multiply(AtA[:,:,2], AtA[:,:,3])
    AtA_1[:,:,3] = np.multiply(AtA[:,:,0], AtA[:,:,5]) - np.multiply(AtA[:,:,2], AtA[:,:,2])
    AtA_1[:,:,4] = -np.multiply(AtA[:,:,0], AtA[:,:,4]) + np.multiply(AtA[:,:,1], AtA[:,:,2])
    AtA_1[:,:,5] = np.multiply(AtA[:,:,0], AtA[:,:,3]) - np.multiply(AtA[:,:,1], AtA[:,:,1])
    
    x1 = np.multiply(AtA[:,:,0], AtA_1[:,:,0])
    x2 = np.multiply(AtA[:,:,1], AtA_1[:,:,1])
    x3 = np.multiply(AtA[:,:,2], AtA_1[:,:,2])    

    detAta = x1+x2+x3
    return AtA_1,detAta

'''
mutiplyIt
'''
def mutiplyIt(AtA_1, Atb):
    result = np.zeros([Atb.shape[0],Atb.shape[1],3])
    result[:,:,0] = np.multiply(AtA_1[:,:,0], Atb[:,:,0]) + np.multiply(AtA_1[:,:,1], Atb[:,:,1]) + np.multiply(AtA_1[:,:,2], Atb[:,:,2])
    result[:,:,1] = np.multiply(AtA_1[:,:,1], Atb[:,:,0]) + np.multiply(AtA_1[:,:,3], Atb[:,:,1]) + np.multiply(AtA_1[:,:,4], Atb[:,:,2])
    result[:,:,2] = np.multiply(AtA_1[:,:,2], Atb[:,:,0]) + np.multiply(AtA_1[:,:,4], Atb[:,:,1]) + np.multiply(AtA_1[:,:,5], Atb[:,:,2])
    return result

In [218]:

'''
Clip out a 2R+1 x 2R+1 window at each point and estimate 
the normal from points within this window. In case the window 
straddles more than a single superpixel, only take points in the 
same superpixel as the centre pixel. Takes about 0.5 second per image.
Does not use ImageStack so is more platform independent, but 
gives different results.
Input: 
depthImage:   in metres
missingMask:  boolean mask of what data was missing
R:            radius of clipping
sc:           to upsample or not
superpixels:  superpixel map to define bounadaries that should
not be straddled

Output: The normal at pixel (x,y) is N(x, y, :)'pt + b(x,y) = 0
N:            Normal field
b:            bias
'''
def computeNormalsSquareSupport(depthImage, missingMask,X,Y,Z, R, sc, cameraMatrix, superpixels):
    #Convert to centi metres
    depthImage = depthImage*100
    XYZf = np.zeros([X.shape[0], X.shape[1], 3])
    XYZf[:,:,0] = X
    XYZf[:,:,1] = Y
    XYZf[:,:,2] = Z
    
    ind = np.where(missingMask == 1)
    X[ind] = np.nan
    Y[ind] = np.nan
    Z[ind] = np.nan
    
    one_Z = np.expand_dims(1/Z, axis=2)
    X_Z = np.divide(X,Z)
    Y_Z = np.divide(Y,Z)
    one = np.copy(Z)
    one[np.isnan(one[:,:]) == False]= 1
    ZZ = np.multiply(Z,Z)
    X_ZZ = np.expand_dims(np.divide(X, ZZ),axis=2)
    Y_ZZ = np.expand_dims(np.divide(Y, ZZ),axis=2)
    
    X_Z_2 =  np.expand_dims(np.multiply(X_Z, X_Z),axis=2)
    XY_Z =  np.expand_dims(np.multiply(X_Z, Y_Z),axis=2)
    Y_Z_2 =  np.expand_dims(np.multiply(Y_Z, Y_Z),axis=2)
    
    
    AtARaw = np.concatenate((X_Z_2, XY_Z, np.expand_dims(X_Z,axis=2), Y_Z_2,
                            np.expand_dims(Y_Z,axis=2), np.expand_dims(one,axis=2)), axis=2)

    AtbRaw = np.concatenate((X_ZZ, Y_ZZ, one_Z), axis=2)
    # with clipping
    AtA = filterItChopOff(np.concatenate((AtARaw, AtbRaw), axis = 2), R, superpixels)
    Atb = AtA[:, :, AtARaw.shape[2]:]
    AtA = AtA[:, :, :AtARaw.shape[2]]
  
    AtA_1, detAtA = invertIt(AtA);

    N = mutiplyIt(AtA_1, Atb);

    divide_fac = np.sqrt(np.sum(np.multiply(N,N), axis=2))

    b = np.divide(-detAtA, divide_fac)
    for i in range(3):
        N[:,:,i] = np.divide(N[:,:,i], divide_fac)    

    # Reorient the normals to point out from the scene.
    SN = np.sign(N[:,:,2])
    SN[SN == 0] = 1
    extend_SN = np.expand_dims(SN, axis= 2)
    extend_SN = np.concatenate((extend_SN, extend_SN, extend_SN), axis= 2)
    N = np.multiply(N, extend_SN)
    b = np.multiply(b, SN)
    sn = np.sign(np.sum(np.multiply(N, XYZf),axis = 2))
    sn[np.isnan(sn)] = 1
    sn[sn == 0] = 1
    N = np.multiply(extend_SN, N)
    b = np.multiply(b, sn)
    return N, b
    

In [219]:
mat = spio.loadmat('biggest.mat', squeeze_me=True)

cameraMatrix= mat['cameraMatrix']
depthImage=mat['depthImage']
missingMask = mat['missingMask']
superpixels = mat['superpixels']
sc = 1
R = 3
X,Y,Z = getPointCloudFromZ(depthImage, cameraMatrix, sc)

In [221]:
N,b  = computeNormalsSquareSupport(depthImage, missingMask,X,Y,Z, R, sc, cameraMatrix, superpixels)
print(N[0,:10,:])
print(b[0,:10])

[[-0.29180164 -0.90448947  0.31104759]
 [-0.86962237 -0.20034107 -0.45124316]
 [-0.90932467 -0.00433428 -0.41606473]
 [-0.94031079  0.09109096 -0.32789946]
 [-0.96085494  0.03290338 -0.27509118]
 [-0.95071713 -0.09120642 -0.29634156]
 [-0.95405501 -0.10451064 -0.28081412]
 [-0.97234814 -0.09572051 -0.21301801]
 [-0.92705035 -0.2110801  -0.30987552]
 [-0.879133   -0.22080242 -0.42234045]]
[-1.03175445  1.05329954  0.8528181  -1.20427396 -1.74378504 -1.93150767
 -2.07043098 -2.50962694 -2.11013845 -1.30103482]


c:\users\menghe\anaconda3\envs\mpytorch\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide
c:\users\menghe\anaconda3\envs\mpytorch\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in true_divide
c:\users\menghe\anaconda3\envs\mpytorch\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in true_divide
c:\users\menghe\anaconda3\envs\mpytorch\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in sign
c:\users\menghe\anaconda3\envs\mpytorch\lib\site-packages\ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in sign


In [61]:
'''
getYDir: get gravity direction
Input: N: HxWx3 normal field
       angleThresh: degrees the threshold for mapping to parallel to gravity and perpendicular to gravity
       num_iter: number of iterations
       y0: initial gravity direction
Output:gravity direction
'''
def getYDirHelper(N, y0, thresh, num_iter):
    dim = N.shape[0] * N.shape[1]    
    nn = np.swapaxes(np.swapaxes(N,0,2),1,2)
    nn = np.reshape(nn,(3, dim))
    # only keep those non-nan
    idx = np.where(np.isnan(nn[0,:]) == False)[0]
    nn = nn[:,idx]
    yDir = y0;
    for i in range(num_iter):
        sim0 = np.dot(yDir.T, nn)
        indF = abs(sim0) > np.cos(thresh)
        indW = abs(sim0) < np.sin(thresh)
        if(len(indF.shape) == 2):
            NF = nn[:, indF[0,:]]
            NW = nn[:, indW[0,:]]
        else:
            NF = nn[:, indF]
            NW = nn[:, indW]
        A = np.dot(NW, NW.T) - np.dot(NF, NF.T)
        b = np.zeros([3,1])
        c = NF.shape[1]
        w,v = np.linalg.eig(A)
        min_ind = np.argmin(w)
        newYDir = v[:,min_ind]
        yDir = newYDir * np.sign(np.dot(yDir.T, newYDir))
    return yDir
def getYDir(N, angleThresh, num_iter, y0):
    y = y0
    for i in range(len(angleThresh)):
        thresh = np.pi*angleThresh[i]/180
        y = getYDirHelper(N, y, thresh, num_iter[i])
    return y
            
    

In [ ]:
'''
processDepthImage
Input: z value in cm, C is intrinsic camera matrix
Output: pc : height x width x 3 , XYZ in real world
        N : height x width x 3 normal vector for each pixel
        yDir: 3x1 y direction(note it is the opposite dir)
        h: height x width height (could be negtive if under camera)for each pixel
        R: 3x3 correction rotation matrix
'''
def processDepthImage(z, missingMask, C):
    ydir_angleThresh = np.array([45,15])
    ydir_iter = np.array([5,5])
    ydir_y0 = np.array([[0,1,0]]).T
    
    normal_patch_size = np.array([3, 10])
    
    X,Y,Z = getPointCloudFromZ(z, C, 1)
    pc = np.concatenate((X,Y,Z), axis=2)
    
    # Compute the normals for this image
    N1,b1 = computeNormalsSquareSupport(z./100, missingMask, normal_patch_size[1], 1, C, np.ones(z.shape))
    N2,b2 = computeNormalsSquareSupport(z./100, missingMask, normal_patch_size[2], 1, C, np.ones(z.shape));
    
    yDir = getYDir(N2, ydir_angleThresh, ydir_iter, ydir_y0)
    y0 = np.array([[0, 1 ,0]]).T
    R = getRMatrix(y0, yDir)
    pcRot = rotatePC(pc, R.T)
    h = -pcRot(:,:,2)
    yMin = prctile(h(:), 0); if(yMin > -90) yMin = -130; end
    h = h-yMin
    return pc, N, yDir, h, R.T
                   